In [1]:
# torch
import torch
import torch.nn as nn
import torch.nn.functional as F

# torch vision
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

# data loader
from torch.utils.data.dataloader import DataLoader

# optimizer
from torch.optim import optimizer

In [2]:
path = '/home/helpme/MNIST'

In [3]:
data_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

In [4]:
training_data = MNIST(
    root=path,
    train=True,
    download=True,
    transform=data_transform
)

test_data = MNIST(
    root=path,
    train=False,
    download=True,
    transform=data_transform
)

100%|██████████| 9912422/9912422 [00:00<00:00, 146815703.08it/s]


Extracting /home/helpme/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /home/helpme/MNIST/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 115038645.61it/s]


Extracting /home/helpme/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /home/helpme/MNIST/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 92176243.81it/s]

Extracting /home/helpme/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/helpme/MNIST/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 8065422.85it/s]

Extracting /home/helpme/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/helpme/MNIST/MNIST/raw



In [ ]:
batch_size = 16

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [ ]:
print('train 데이터 개수 : ', len(train_dataloader))
print('test 데이터 개수: ', len(test_dataloader))
print()
print('=========data shape=========')

for x,y in test_dataloader:
  print('Shape of x{N,C,H,W}', x.shape)
  print('Shape of y:', y.shape, y.dtype)
  break

train 데이터 개수 :  3750
test 데이터 개수:  10000

=========data shape=========
Shape of x{N,C,H,W} torch.Size([1, 1, 32, 32])
Shape of y: torch.Size([1]) torch.int64


# 모델 만들기

In [ ]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
    self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

    self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
    self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

    self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0)

    # fully connected
    self.fc1 = nn.Linear(in_features=120, out_features=84)
    self.fc2 = nn.Linear(in_features=84, out_features=10)

  def forward(self, x):
    x = torch.tanh(self.conv1(x))
    x = self.pool1(x)

    x = torch.tanh(self.conv2(x))
    x = self.pool2(x)

    x = torch.tanh(self.conv3(x))

    x = x.view(-1, 120) # flatten

    x = torch.tanh(self.fc1(x)) # f1
    x = self.fc2(x)

    return x

In [ ]:
model = LeNet()

In [ ]:
# model = LeNet_with_sequential()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [ ]:
model.to(device)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
from torchsummary import summary
summary(model, input_size=(1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         AvgPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         AvgPool2d-4             [-1, 16, 5, 5]               0
            Conv2d-5            [-1, 120, 1, 1]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
----------------------------------------------------------------


In [ ]:
class LeNet_sequential(nn.Module):
  def __init__(self):
    super(LeNet_sequential, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0),
        nn.Tanh()
    )

    # fully connected
    self.layer4 = nn.Sequential(
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh()
    )

    self.layer5 = nn.Sequential(
        nn.Linear(in_features=84, out_features=10)
    )

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)

    x = x.view(-1, 120) # flatten

    x = self.layer4(x)
    x = self.layer5(x)
    return x


In [ ]:
model = LeNet_sequential()
model.to(device)

LeNet_sequential(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
  )
  (layer4): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Tanh()
  )
  (layer5): Sequential(
    (0): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
criterion.to(device=device)

CrossEntropyLoss()

In [ ]:
total_step = len(train_dataloader)

In [ ]:
total_step

3750

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_dataloader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 400 == 0:
      print('Epoch [{}/{}, Step [{}/{}], Loss: {:.4f}'
                            .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10, Step [400/3750], Loss: 2.2422
Epoch [1/10, Step [800/3750], Loss: 1.6619
Epoch [1/10, Step [1200/3750], Loss: 0.6446
Epoch [1/10, Step [1600/3750], Loss: 0.6360
Epoch [1/10, Step [2000/3750], Loss: 0.3413
Epoch [1/10, Step [2400/3750], Loss: 0.3097
Epoch [1/10, Step [2800/3750], Loss: 0.2329
Epoch [1/10, Step [3200/3750], Loss: 0.1573
Epoch [1/10, Step [3600/3750], Loss: 0.0613
Epoch [2/10, Step [400/3750], Loss: 0.6545
Epoch [2/10, Step [800/3750], Loss: 0.2661
Epoch [2/10, Step [1200/3750], Loss: 0.5742
Epoch [2/10, Step [1600/3750], Loss: 0.1883
Epoch [2/10, Step [2000/3750], Loss: 0.0405
Epoch [2/10, Step [2400/3750], Loss: 0.0746
Epoch [2/10, Step [2800/3750], Loss: 0.1064
Epoch [2/10, Step [3200/3750], Loss: 0.2401
Epoch [2/10, Step [3600/3750], Loss: 0.1974
Epoch [3/10, Step [400/3750], Loss: 0.0742
Epoch [3/10, Step [800/3750], Loss: 0.2165
Epoch [3/10, Step [1200/3750], Loss: 0.3570
Epoch [3/10, Step [1600/3750], Loss: 0.0957
Epoch [3/10, Step [2000/3750], Loss: 0

In [ ]:
# testing
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_dataloader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: {}%'.format(100 * correct / total))

# Accuracy of the network on the 10000 test images: 98.39%

Accuracy of the network on the 10000 test images: 98.57%
